In [35]:
from pathlib import Path
import numpy as np
import pandas as pd

In [36]:
meta = pd.read_csv(Path.cwd() / 'data' / 'meta.csv', parse_dates=['first_include'], date_format='%Y-%m-%d')

In [37]:
df = pd.read_csv(Path.cwd() / 'data' / 'historical_prices_monthly_stat.csv')
df = df.sort_values(['_code', '_year', '_month'], ascending=True).reset_index(drop=True)

In [38]:
# bins = [-np.inf, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, -0.05, -0.01, 0]
bins = [-np.inf, -0.4, -0.1, -0.01, 0]
labels = [f'({bins[i-1]}, {bins[i]}]' for i, _ in enumerate(bins) if i > 0]
df['monthly_high_end_rtn_category'] = pd.cut(df['monthly_high_end_rtn'], bins=bins, labels=labels).astype(str)
df['before_monthly_high_end_rtn'] = df.groupby('_code', as_index=False)['monthly_high_end_rtn'].shift(1)
df['before_monthly_high_end_rtn_category'] = df.groupby('_code', as_index=False)['monthly_high_end_rtn_category'].shift(1)

In [39]:
df

,_code,_year,_month,monthly_rtn,monthly_high_end_rtn,monthly_start_high_rtn,daily_rtn_avg,daily_vola,daily_dvola,monthly_mdd,monthly_high_end_rtn_category,before_monthly_high_end_rtn,before_monthly_high_end_rtn_category
0,AN8068571086,2014,4,0.036648,-0.006457,0.043385,0.001837,0.008701,0.005368,0.019390,"(-0.01, 0]",NaN,NaN
1,AN8068571086,2014,5,0.032143,0.000000,0.032143,0.001611,0.007609,0.006297,0.023361,"(-0.01, 0]",-0.006457,"(-0.01, 0]"
2,AN8068571086,2014,6,0.135117,0.000000,0.135117,0.006489,0.016904,0.006522,0.019861,"(-0.01, 0]",0.000000,"(-0.01, 0]"
3,AN8068571086,2014,7,-0.080272,-0.080272,0.000000,-0.003937,0.009125,0.005828,0.079334,"(-0.1, -0.01]",0.000000,"(-0.01, 0]"
4,AN8068571086,2014,8,0.015655,-0.012608,0.028624,0.000832,0.010711,0.007118,0.034285,"(-0.1, -0.01]",-0.080272,"(-0.1, -0.01]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271924,US98980L1017,2023,11,0.129935,-0.001766,0.131934,0.006280,0.017968,0.010555,0.037149,"(-0.01, 0]",-0.144487,"(-0.4, -0.1]"
271925,US98980L1017,2023,12,0.023047,-0.030993,0.055769,0.001368,0.018804,0.011594,0.043800,"(-0.1, -0.01]",-0.001766,"(-0.01, 0]"
271926,US98980L1017,2024,1,-0.065654,-0.083026,0.018944,-0.003228,0.018416,0.012142,0.083026,"(-0.1, -0.01]",-0.030993,"(-0.1, -0.01]"
271927,US98980L1017,2024,2,0.090671,0.000000,0.090671,0.004831,0.023420,0.009059,0.066667,"(-0.01, 0]",-0.083026,"(-0.1, -0.01]"


In [41]:
df = pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby('before_monthly_high_end_rtn_category')[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std'])

In [42]:
df

monthly_rtn                      \
                                           count      mean    median   
before_monthly_high_end_rtn_category                                   
(-0.01, 0]                                 59835  0.009278  0.000000   
(-0.1, -0.01]                             155816  0.006672  0.001951   
(-0.4, -0.1]                               53086  0.010608  0.005861   
(-inf, -0.4]                                 603  0.195009  0.065089   

                                                                     \
                                           min        max       std   
before_monthly_high_end_rtn_category                                  
(-0.01, 0]                           -0.909001  25.681667  0.168326   
(-0.1, -0.01]                        -0.871383   4.645161  0.101974   
(-0.4, -0.1]                         -0.998375  17.840580  0.166740   
(-inf, -0.4]                         -0.924103  13.900000  0.877699   

                                     monthly_start_high_rtn            \
                                                      count      mean   
before_monthly_high_end_rtn_category                                    
(-0.01, 0]                                            59835  0.065984   
(-0.1, -0.01]                                        155816  0.065521   
(-0.4, -0.1]                                          53086  0.096477   
(-inf, -0.4]                                            603  0.441567   

                                                     ... daily_rtn_avg  \
                                        median  min  ...        median   
before_monthly_high_end_rtn_category                 ...                 
(-0.01, 0]                            0.037415  0.0  ...      0.000139   
(-0.1, -0.01]                         0.043201  0.0  ...      0.000261   
(-0.4, -0.1]                          0.064935  0.0  ...      0.000624   
(-inf, -0.4]                          0.205253  0.0  ...      0.005433   

                                                                     \
                                           min        max       std   
before_monthly_high_end_rtn_category                                  
(-0.01, 0]                           -0.057508  17.341999  0.071238   
(-0.1, -0.01]                        -0.081854   0.121692  0.005109   
(-0.4, -0.1]                         -0.076269   0.627274  0.007988   
(-inf, -0.4]                         -0.056966   7.107188  0.314708   

                                     monthly_mdd                           \
                                           count      mean    median  min   
before_monthly_high_end_rtn_category                                        
(-0.01, 0]                                 59825  0.075424  0.060994  0.0   
(-0.1, -0.01]                             155807  0.079167  0.064567  0.0   
(-0.4, -0.1]                               53084  0.115639  0.090909  0.0   
(-inf, -0.4]                                 602  0.249514  0.204446  0.0   

                                                          
                                           max       std  
before_monthly_high_end_rtn_category                      
(-0.01, 0]                            0.997230  0.059385  
(-0.1, -0.01]                         0.887529  0.057070  
(-0.4, -0.1]                          0.998857  0.088719  
(-inf, -0.4]                          0.985333  0.176604  

[4 rows x 24 columns]

In [22]:
import plotly.graph_objects as go
import plotly.express as px

orange = px.colors.sequential.Oranges
orange.append('rgb(245,130,32)')

fig = go.Figure()

fig.add_trace(go.Bar(
    y=df[df.columns[0]],
    x=df[df.columns[0]].index,
    marker = {
        'color': 'rgb(245,130,32)',
        'opacity': [0.5 + i / (2 * (len(df[df.columns[0]].index)-1)) for i in range(len(df[df.columns[0]].index)-1)]
    }
))

fig.show()

In [ ]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['country', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std'])

In [ ]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['gics_sector', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std'])

In [ ]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['gics_sector', 'gics_industry_group', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std'])

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure()
# for lb in labels:
#     fig.add_trace(go.Box(x=df[df['before_monthly_high_end_rtn_category'] == lb]['monthly_start_high_rtn'], name=lb, boxpoints=False))
# fig.show()

In [ ]:
df['before_monthly_high_end_rtn_category'].value_counts(dropna=False)

In [ ]:
mp = pd.read_csv(Path.cwd() / 'data' / 'gics_mapper.csv', dtype='str')
mp.columns = ['value', 'label']

In [ ]:
v = '0'
filtered_meta = meta
filtered_meta = meta if v == '0' else meta[meta['country'] == v]

In [ ]:
filtered_meta

In [ ]:
df = pd.read_csv(Path.cwd() / 'data' / 'historical_prices_monthly_stat.csv')
df = df.sort_values(['_code', '_year', '_month'], ascending=True).reset_index(drop=True)

In [ ]:
df